In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# scripts/explore_coco.py (Kaggle version)
import os, json, random
from pycocotools.coco import COCO
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import cv2
from collections import Counter
import pandas as pd

# ---- USER CONFIG ----
BASE_DIR = "/kaggle/input/coco-2017-dataset/coco2017"
DATA_DIR = BASE_DIR
ANN_DIR = os.path.join(DATA_DIR, "annotations")
TRAIN_ANN = os.path.join(ANN_DIR, "instances_train2017.json")
VAL_ANN = os.path.join(ANN_DIR, "instances_val2017.json")
IMG_DIR_TRAIN = os.path.join(DATA_DIR, "train2017")
IMG_DIR_VAL = os.path.join(DATA_DIR, "val2017")
OUTPUT_DIR = "/kaggle/working/exploration_outputs"

SAMPLE_COUNT = 8

# ✅ Selected 24 classes for autonomous-driving dataset
SELECTED_CLASSES = [
    "person", "bicycle", "car", "motorcycle", "bus", "truck", "train",
    "traffic light", "stop sign", "fire hydrant", "bench", "parking meter",
    "umbrella", "backpack", "handbag", "tie", "cell phone", "dog", "cat",
    "horse", "bird", "skateboard", "boat", "suitcase"
]

os.makedirs(OUTPUT_DIR, exist_ok=True)

# --------------------------------------------------------
def explore(ann_file, img_dir, split_name="train"):
    print(f"\n=== Exploring {split_name.upper()} split ===")
    coco = COCO(ann_file)
    cats = coco.loadCats(coco.getCatIds())
    cat_id_to_name = {c['id']: c['name'] for c in cats}
    selected_cat_ids = coco.getCatIds(catNms=SELECTED_CLASSES)

    # --- Basic statistics ---
    img_ids = coco.getImgIds()
    images = coco.loadImgs(img_ids)
    ann_ids = coco.getAnnIds()
    anns = coco.loadAnns(ann_ids)

    widths = [img['width'] for img in images]
    heights = [img['height'] for img in images]
    class_counts = Counter()
    for a in anns:
        if a['category_id'] in selected_cat_ids:
            class_counts[cat_id_to_name[a['category_id']]] += 1

    objs_per_image = [len(coco.getAnnIds(imgIds=img['id'], catIds=selected_cat_ids)) for img in images]

    summary = {
        "split": split_name,
        "num_images": len(images),
        "num_annotations_total": len(anns),
        "num_annotations_selected": sum(class_counts.values()),
        "image_width_mean": np.mean(widths),
        "image_height_mean": np.mean(heights),
        "avg_objects_selected_per_image": np.mean(objs_per_image)
    }
    pd.DataFrame([summary]).to_csv(os.path.join(OUTPUT_DIR, f"summary_{split_name}.csv"), index=False)
    print("Summary:", summary)

    # --- Visualization section ---
    sns.set_style("whitegrid")

    # 1. Class distribution
    plt.figure(figsize=(10,6))
    classes = list(class_counts.keys())
    counts = [class_counts[c] for c in classes]
    sns.barplot(x=counts, y=classes, palette="crest")
    plt.title(f"Class Distribution - {split_name}")
    plt.xlabel("Object count")
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, f"class_distribution_{split_name}.png"))
    plt.close()

    # 2. Objects per image histogram
    plt.figure(figsize=(7,5))
    plt.hist(objs_per_image, bins=30, color="royalblue")
    plt.xlabel("Objects per image")
    plt.ylabel("Number of images")
    plt.title(f"Objects per Image Histogram ({split_name})")
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, f"objects_per_image_{split_name}.png"))
    plt.close()

    # 3. Image dimension distribution
    plt.figure(figsize=(6,6))
    plt.scatter(widths, heights, s=10, alpha=0.4)
    plt.xlabel("Width")
    plt.ylabel("Height")
    plt.title(f"Image Resolution Scatter ({split_name})")
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, f"image_resolution_{split_name}.png"))
    plt.close()

    # --- Sample image visualization ---
    print("Generating example annotated images...")
    sample_img_ids = random.sample(img_ids, min(SAMPLE_COUNT, len(img_ids)))
    for i, img_id in enumerate(sample_img_ids):
        img_info = coco.loadImgs(img_id)[0]
        img_path = os.path.join(img_dir, img_info['file_name'])
        if not os.path.exists(img_path):
            continue
        img = cv2.imread(img_path)
        anns_ids = coco.getAnnIds(imgIds=img_id, catIds=selected_cat_ids)
        anns_sel = coco.loadAnns(anns_ids)
        for ann in anns_sel:
            x, y, w, h = map(int, ann['bbox'])
            cat_name = cat_id_to_name[ann['category_id']]
            cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)
            cv2.putText(img, cat_name, (x, max(15,y-5)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 1)
        out_path = os.path.join(OUTPUT_DIR, f"sample_{split_name}_{i}.jpg")
        cv2.imwrite(out_path, img)

    # --- Automatic insights ---
    print("\n📊 Key Insights:")
    top5 = class_counts.most_common(5)
    print(f"Top 5 most common objects: {top5}")
    rare5 = class_counts.most_common()[-5:]
    print(f"Least common objects: {rare5}")
    print(f"Average objects per image: {summary['avg_objects_selected_per_image']:.2f}")
    print(f"Average image size: {summary['image_width_mean']:.0f}×{summary['image_height_mean']:.0f}")

    print("\n💡 Analysis Q&A:")
    print("Q1: Which objects dominate the dataset?")
    print(f"A1: {top5[0][0]} appears most frequently, suggesting focus on urban scenes.")
    print("Q2: Are there underrepresented classes?")
    print(f"A2: The rarest are {', '.join([c for c,_ in rare5])}, consider augmenting them.")
    print("Q3: How dense are scenes?")
    print(f"A3: On average {summary['avg_objects_selected_per_image']:.1f} relevant objects per image.")
    print("Q4: Are image sizes consistent?")
    print(f"A4: Widths range {min(widths)}–{max(widths)}, heights {min(heights)}–{max(heights)}.")
    print("Q5: What do examples show?")
    print("A5: Sample images saved with bounding boxes illustrate diversity in object placement and scale.\n")

    return summary, class_counts


if __name__ == "__main__":
    print("Starting Dataset Exploration ...")
    explore(TRAIN_ANN, IMG_DIR_TRAIN, "train")
    explore(VAL_ANN, IMG_DIR_VAL, "val")
    print(f"\n✅ Exploration finished. Check visual reports in:\n{OUTPUT_DIR}")


In [ ]:
# scripts/filter_and_convert_kaggle.py
import os, json, shutil
from pycocotools.coco import COCO
from tqdm import tqdm
from PIL import Image
import numpy as np
import yaml

# ==========================================================
# ✅ USER CONFIG (Kaggle Version of Your Original Script)
# ==========================================================
BASE_DIR = "/kaggle/working"
DATA_DIR = "/kaggle/input/coco-2017-dataset/coco2017"
ANN_DIR = os.path.join(DATA_DIR, "annotations")

TRAIN_ANN = os.path.join(ANN_DIR, "instances_train2017.json")
VAL_ANN = os.path.join(ANN_DIR, "instances_val2017.json")

IMG_DIR_TRAIN = os.path.join(DATA_DIR, "train2017")
IMG_DIR_VAL = os.path.join(DATA_DIR, "val2017")

OUT = os.path.join(BASE_DIR, "outputs", "filtered_dataset")
IMG_SIZE = 640  # final image size

# ✅ Debug: Check paths
print("TRAIN_ANN exists:", os.path.exists(TRAIN_ANN))
print("VAL_ANN exists:", os.path.exists(VAL_ANN))
print("IMG_DIR_TRAIN exists:", os.path.exists(IMG_DIR_TRAIN))
print("IMG_DIR_VAL exists:", os.path.exists(IMG_DIR_VAL))

# ==========================================================
# COCO Classes of Interest
# ==========================================================
SELECTED_CLASSES = [
    "person", "bicycle", "car", "motorcycle", "bus", "truck", "train",
    "traffic light", "stop sign", "fire hydrant", "bench", "parking meter",
    "umbrella", "backpack", "handbag", "tie", "cell phone", "dog", "cat",
    "horse", "bird", "skateboard", "boat", "suitcase"
]

SPLIT_MAP = {
    "train": (TRAIN_ANN, IMG_DIR_TRAIN),
    "val": (VAL_ANN, IMG_DIR_VAL)
}

# Create output folders
os.makedirs(OUT, exist_ok=True)
for s in ["train", "val"]:
    os.makedirs(os.path.join(OUT, s, "images"), exist_ok=True)
    os.makedirs(os.path.join(OUT, s, "labels"), exist_ok=True)


# ==========================================================
# Helper Functions
# ==========================================================
def build_cat_mapping(coco, selected_names):
    selected_cat_ids = coco.getCatIds(catNms=selected_names)
    print("Selected cat IDs:", selected_cat_ids)
    if not selected_cat_ids:
        print("⚠️ Warning: No matching categories found! Check class names.")
    mapping = {cid: i for i, cid in enumerate(selected_cat_ids)}
    return mapping, selected_cat_ids


def process_split(split):
    ann_file, img_dir = SPLIT_MAP[split]
    print(f"\n=== Processing {split} split ===")
    print("Annotation file:", ann_file)
    print("Exists?", os.path.exists(ann_file))
    coco = COCO(ann_file)

    mapping, selected_cat_ids = build_cat_mapping(coco, SELECTED_CLASSES)
    selected_cat_names = [coco.loadCats([cid])[0]['name'] for cid in selected_cat_ids]

    test_car_id = coco.getCatIds(catNms=["car"])
    test_car_imgs = coco.getImgIds(catIds=test_car_id)
    print("Debug - Car ID:", test_car_id, "| Images found:", len(test_car_imgs))

    img_ids = list(set().union(*[coco.getImgIds(catIds=[cid]) for cid in selected_cat_ids]))
    print(f"{split}: {len(img_ids)} images with selected classes")

    for img_id in tqdm(img_ids):
        img_meta = coco.loadImgs(img_id)[0]
        fname = img_meta['file_name']
        src_path = os.path.join(img_dir, fname)
        if not os.path.exists(src_path):
            continue

        im = Image.open(src_path).convert('RGB')
        orig_w, orig_h = im.size
        im_resized = im.resize((IMG_SIZE, IMG_SIZE))
        out_img_path = os.path.join(OUT, split, "images", fname)
        im_resized.save(out_img_path, quality=95)

        ann_ids = coco.getAnnIds(imgIds=img_id, catIds=selected_cat_ids, iscrowd=None)
        anns = coco.loadAnns(ann_ids)

        yolo_lines = []
        for ann in anns:
            coco_cat_id = ann['category_id']
            if coco_cat_id not in mapping:
                continue
            new_class = mapping[coco_cat_id]
            x, y, w, h = ann['bbox']
            x_c = (x + w / 2.0) / orig_w
            y_c = (y + h / 2.0) / orig_h
            w /= orig_w
            h /= orig_h
            yolo_lines.append(f"{new_class} {x_c:.6f} {y_c:.6f} {w:.6f} {h:.6f}")

        label_path = os.path.join(OUT, split, "labels", os.path.splitext(fname)[0] + ".txt")
        with open(label_path, "w") as f:
            if len(yolo_lines) > 0:
                f.write("\n".join(yolo_lines))

    inv_map = {v: k for k, v in mapping.items()}
    names = [coco.loadCats([inv_map[i]])[0]['name'] for i in range(len(inv_map))]
    return names


# ==========================================================
# Main Execution
# ==========================================================
if __name__ == "__main__":
    names = None
    for s in ["val"]:  # change to ["train", "val"] to process both
        names = process_split(s)

    data_yaml = {
        'train': os.path.abspath(os.path.join(OUT, "train", "images")),
        'val': os.path.abspath(os.path.join(OUT, "val", "images")),
        'test': os.path.abspath(os.path.join(OUT, "val", "images")),
        'nc': len(names),
        'names': names
    }

    yaml_path = os.path.join(BASE_DIR, "data.yaml")
    with open(yaml_path, "w") as f:
        yaml.dump(data_yaml, f)

    print("\n✅ Conversion complete!")
    print("data.yaml created at:", yaml_path)
    print("Classes:", names)


In [ ]:
!pip install ultralytics==8.2.77


In [ ]:
from ultralytics import YOLO
import os


In [ ]:
!ls /kaggle/working/outputs/filtered_dataset/train/images | head
!ls /kaggle/working/outputs/filtered_dataset/val/images | head
!cat /kaggle/working/data.yaml


In [ ]:
model = YOLO("yolov8l.pt")  


In [ ]:
model.train(
    data="/kaggle/working/data.yaml",
    epochs=120,
    imgsz=800,
    batch=8,
    workers=2,                 # keep low for Kaggle stability
    optimizer="AdamW",
    lr0=0.0005,
    cos_lr=True,
    augment=True,
    dropout=0.2,
    patience=10,               # early stop if no improvement
    device=0,                  # use GPU 0
    project="yolo8_coco24_runs",
    name="yolo8_coco24_highacc",
    cache=True,
)


In [ ]:
# === Step 1: Disable W&B (prevents hangs) ===
import os
os.environ["WANDB_MODE"] = "disabled"

# === Step 2: Install/upgrade ultralytics ===
!pip install -U ultralytics > /dev/null

# === Step 3: Verify GPU ===
!nvidia-smi

# === Step 4: Train YOLOv8 ===
from ultralytics import YOLO

model = YOLO("yolov8l.pt")  # or "yolov8x.pt" for extra accuracy (slower)

model.train(
    data="/kaggle/working/data.yaml",
    epochs=120,
    imgsz=800,
    batch=8,
    workers=2,                 # keep low for Kaggle stability
    optimizer="AdamW",
    lr0=0.0005,
    cos_lr=True,
    augment=True,
    dropout=0.2,
    patience=10,               # early stop if no improvement
    device=0,                  # use GPU 0
    project="yolo8_coco24_runs",
    name="yolo8_coco24_highacc",
    cache=True,
)

